In [26]:
import os
import glob
import pandas as pd
import random
from pyspark.sql import SparkSession
print(os.getcwd())

/usr/local/share_storages/lab


In [24]:
paths = glob.glob("/usr/local/share_storages/data/image/dog-cat/*.jpeg")
random_path = [random.choice(paths) for i in range(3000)]
df = pd.DataFrame(random_path, columns=['path'])
df

,path
0,/usr/local/share_storages/data/image/dog-cat/a...
1,/usr/local/share_storages/data/image/dog-cat/a...
2,/usr/local/share_storages/data/image/dog-cat/a...
3,/usr/local/share_storages/data/image/dog-cat/c...
4,/usr/local/share_storages/data/image/dog-cat/a...
...,...
2995,/usr/local/share_storages/data/image/dog-cat/a...
2996,/usr/local/share_storages/data/image/dog-cat/a...
2997,/usr/local/share_storages/data/image/dog-cat/a...
2998,/usr/local/share_storages/data/image/dog-cat/2...


In [28]:
ss  = SparkSession.builder.master('spark://spark-master:7077') \
                  .appName("test") \
                  .config("spark.executor.memory", "2g") \
                  .config("spark.driver.memory", "2g") \
                  .config("spark.executor.cores", "1") \
                  .config("spark.driver.cores", "1") \
                  .getOrCreate()
sc = ss.sparkContext
sc

<SparkContext master=spark://spark-master:7077 appName=test>